In [2]:
! pip install openai

   ---------------------------------------- 0.0/383.7 kB ? eta -:--:--
   ---- ----------------------------------- 41.0/383.7 kB 1.9 MB/s eta 0:00:01
   ------ -------------------------------- 61.4/383.7 kB 812.7 kB/s eta 0:00:01
   ----------- -------------------------- 112.6/383.7 kB 930.9 kB/s eta 0:00:01
   -------------- ----------------------- 143.4/383.7 kB 847.9 kB/s eta 0:00:01
   ------------------------ --------------- 235.5/383.7 kB 1.1 MB/s eta 0:00:01
   ----------------------------------- ---- 337.9/383.7 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 383.7/383.7 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.4 kB ? eta -:--:--
   ------------------------------------- -- 71.7/76.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 76.4/76.4 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.0 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.0 kB 3.8 MB/s eta 0:00

In [3]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'openai')
openai = OpenAI()

# openai = OpenAI(api_key="your-key-here")

In [5]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out

ed = Website("https://medium.com/global-ai-hub/langchain-temelleri-10963adefccd")
print(ed.title)
print(ed.text)

LangChain Temelleri. LangChain Nedir? | by Defne Buse Çelik | Global AI Hub | Oct, 2024 | Medium
Open in app
Sign up
Sign in
Write
Sign up
Sign in
LangChain Temelleri
Defne Buse Çelik
·
Follow
Published in
Global AI Hub
·
10 min read
·
Just now
--
Share
LangChain Nedir?
LangChain, doğal dil işleme projelerinin esnekliğini ve gücünü artıran yenilikçi bir çerçevedir. Büyük dil modelleri (LLM) ile derin etkileşim sağlayarak metin oluşturma, sorgulama, analiz ve veri kaynaklarıyla entegrasyon gibi çok yönlü görevler için optimize edilmiştir.
LangChain, süreçleri modüler bir yapıya dönüştürmektedir ve geliştiricilerin adım adım zincirler oluşturmalarını sağlamaktadır. Bu sayede kullanıcıların karmaşık dil işleme görevlerini tek bir çerçeve içinde ve organize bir halde ölçeklenebilir çözümleri kolaylaştırmalarını sağlar.
LangChain, LLM uygulamalarının gelişimini hızlandırmak için tasarlanmıştır. LangChain, yazılım geliştiricilerinin harici bileşenler ve veri kaynaklarıyla LLM uygulamalarını 

## Types of prompts



**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [9]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [23]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://medium.com/global-ai-hub/langchain-temelleri-10963adefccd")

In [25]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://medium.com/global-ai-hub/langchain-temelleri-10963adefccd")